In [18]:
!pip install transformers datasets pdfplumber scikit-learn -q


In [20]:
import os
from huggingface_hub import login

login(token=os.getenv("hftoken"))


In [21]:
import pdfplumber
import os

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Upload file in Colab
from google.colab import files
uploaded = files.upload()

file_name = list(uploaded.keys())[0]

# Check if the uploaded file is a PDF
if file_name.lower().endswith('.pdf'):
    doc_text = extract_text_from_pdf(file_name)
    print(doc_text[:1000])
else:
    print(f"Error: The uploaded file '{file_name}' is not a PDF. Please upload a PDF file.")

Saving Medical_Claim_Form_John_Doe.pdf to Medical_Claim_Form_John_Doe.pdf
Medical Claim Form
Patient Name: John Doe
Age: 35
Gender: Male
Policy Number: POL123456789
Claim Number: CLM987654321
Date of Treatment: 2025-07-15
Hospital Name: City Care Hospital
Diagnosis: Acute Appendicitis
Treatment Provided: Appendectomy Surgery
Total Claim Amount (INR): INR 45,000
Doctor's Name: Dr. Anjali Sharma
Remarks: Patient discharged in stable condition.
Date of Submission: 2025-07-20
Page 1



In [22]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Define possible claim types and priorities
labels = ["Vehicle Damage", "Medical Claim", "Property Damage", "Fraud", "Low Priority", "High Priority"]

result = classifier(doc_text, candidate_labels=labels, multi_label=True)
print(result)


Device set to use cpu


{'sequence': "Medical Claim Form\nPatient Name: John Doe\nAge: 35\nGender: Male\nPolicy Number: POL123456789\nClaim Number: CLM987654321\nDate of Treatment: 2025-07-15\nHospital Name: City Care Hospital\nDiagnosis: Acute Appendicitis\nTreatment Provided: Appendectomy Surgery\nTotal Claim Amount (INR): INR 45,000\nDoctor's Name: Dr. Anjali Sharma\nRemarks: Patient discharged in stable condition.\nDate of Submission: 2025-07-20\nPage 1\n", 'labels': ['Medical Claim', 'High Priority', 'Low Priority', 'Fraud', 'Vehicle Damage', 'Property Damage'], 'scores': [0.987032949924469, 0.5258276462554932, 0.07911761105060577, 0.008453846909105778, 0.007863410748541355, 0.002755402820184827]}


In [23]:
def route_claim(predictions):
    claim_type = None
    priority = None

    for label, score in zip(predictions['labels'], predictions['scores']):
        if score > 0.7:
            if "Priority" in label:
                priority = label
            else:
                claim_type = label

    if claim_type and priority:
        print(f"Routing to {claim_type} department with {priority} urgency.")
    else:
        print("Manual review required.")


In [24]:
def policy_check(text):
    rules = {
        "Vehicle Damage": ["accident", "vehicle", "license"],
        "Medical Claim": ["hospital", "treatment", "injury"],
        "Fraud": ["fake", "scam", "false"]
    }

    flagged = []
    for claim_type, keywords in rules.items():
        if any(keyword in text.lower() for keyword in keywords):
            flagged.append(claim_type)

    if not flagged:
        return "No policy violations"
    return f"Check for policy: {', '.join(flagged)}"

print(policy_check(doc_text))


Check for policy: Medical Claim


In [25]:
def process_claim(file_path):
    text = extract_text_from_pdf(file_path)
    classification = classifier(text, candidate_labels=labels, multi_label=True)
    route_claim(classification)
    print(policy_check(text))

process_claim(file_name)


Manual review required.
Check for policy: Medical Claim
